In [ ]:
import os

In [1]:
param_path = '/pscratch/sd/j/joeschm/NSXTU_discharges/132588/r_0.736_q4_MTM_mode/convergence_check/nz0_hpyz_edgeopt_scans/parameters'

In [96]:
import os

def read_file(file_name):
    file_path = os.path.join(param_path, file_name)
    with open(file_path, 'r') as file:
        file_content = file.read()
    return file_content


def remove_scan_line(file_content):
    processed_lines = []

    lines = file_content.split('\n')
    for line in lines:
        if '!scan' in line:
            index = line.index('!scan')
            processed_line = line[:index]
            processed_lines.append(processed_line.rstrip())
        else:
            processed_lines.append(line.rstrip())
    return '\n'.join(processed_lines)






def replace_scan_to_line(file_content, input_string, values, scan_type='!scanlist'):
    scan_added = False
    processed_lines = []

    lines = file_content.split('\n')
    for line in lines:
        if '=' in line and line.split('=')[0].strip() == input_string:
            line_parts = line.split('!')
            if len(line_parts) > 1:
                # If there's a comment, insert the scan before it
                processed_line = line_parts[0].rstrip() + ' ' + scan_type + ': ' + ', '.join(map(str, values)) + ' !' + '!'.join(line_parts[1:])
            else:
                # If there's no comment, just append the scan at the end
                processed_line = line.rstrip() + ' ' + scan_type + ': ' + ', '.join(map(str, values))
            processed_lines.append(processed_line)
            scan_added = True
        else:
            processed_lines.append(line.rstrip())
    return '\n'.join(processed_lines), scan_added




def add_scan_to_line(file_content, input_string, values, scan_type='!scanlist'):
    file_lines = []
    sections = []
    current_section = None


    lines = file_content.split('\n')
    for line in lines:
        file_lines.append(line)
        if line.strip().startswith('&'):
            current_section = line.strip()
            sections.append(current_section)
            
    
    
    section_indices = [f"{index}: {section}" for index, section in enumerate(sections)]
    print("Sections:")
    print("\n".join(section_indices))
    
    section = input('Enter the index of the section where you want to add the scan: ')
    try:
        section_index = int(section)
        if section_index < 0 or section_index >= len(sections):
            raise ValueError("Invalid section index")
        else:
            # Find the index of the section in the processed_lines list
            section_line_index = file_lines.index(sections[section_index])
            # Insert the key with a default value of 0 right after the section divider
            processed_line = f'{input_string} = 0' + ' ' + scan_type + ': ' + ', '.join(map(str, values))
            file_lines.insert(section_line_index + 1, processed_line)
    except ValueError:
        raise ValueError("Invalid section index")

    return '\n'.join(file_lines)

In [98]:
file_content = read_file(param_path)
file_mod_content = remove_scan_line(file_content)


scan_dict = {
    'nz0': [1,2,3],
    'nv0': [1,2,3],
    'kymin': [20,30,40],
    'theta0': [10,20]
}

key_dict = 'nz0'
# add_scan_to_line(param_path, 'nz0', [1,2,3]).split('\n')

out, switch = replace_scan_to_line(file_mod_content, key_dict, scan_dict[key_dict])

if not switch:
    out = add_scan_to_line(file_mod_content, key_dict, scan_dict[key_dict])

# output.split('\n')
    
print(out)

&parallelization
!n_procs: number of mpi processes per corresponding
!         dimension (automatic parallelization if <1)
!For follow-up runs, you may set the mapping taken
!from the (output) parameters.dat to speed up the
!initialization
n_procs_s = 1
n_procs_v = 2
n_procs_w = 4
n_procs_x = 1
n_procs_y = 1
n_procs_z = 32
n_procs_sim = 256

n_parallel_sims = 18
/

&box
n_spec =   3   !
nx0    =   7  !usually 128 will be enough for nonlinear runs,but if the heat flux spectrum shows the heat flux does not approach zero at large kx, then you need more
nky0   =   1  ! 1 for linear runs. Nonzero for nonlinear runs. This depends on the linear simulations, larger number if the spectrum is wide
nz0    =   128 !scanlist: 1, 2, 3 !pedestal top 64~128, mid pedestal >256
nv0    =   36   !
nw0    =   8    ! 8 is usually fine, but 16 is better if nky and nx are large

kymin = 0.2

lv    =   3.00000   !
lw    =   9.00000   !
lx    =   4.87775   ! box size, larger lx corresponds to smaller kxmin

ada